In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_curve, f1_score
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load the extracted audio features from CSV
data = pd.read_csv('DATASET-LA.csv')

# Assuming your CSV has columns like 'feature_1', 'feature_2', ..., 'label'
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for SqueezeNet
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the SqueezeNet model
def fire_module(x, squeeze, expand):
    y = layers.Conv1D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y1 = layers.Conv1D(filters=expand // 2, kernel_size=1, activation='relu', padding='same')(y)
    y3 = layers.Conv1D(filters=expand // 2, kernel_size=3, activation='relu', padding='same')(y)
    return layers.concatenate([y1, y3])

def build_squeezenet(input_shape, num_classes):
    x = layers.Input(shape=input_shape)

    y = layers.Conv1D(64, kernel_size=3, activation='relu', padding='same')(x)
    y = layers.MaxPooling1D(pool_size=2)(y)

    y = fire_module(y, squeeze=16, expand=64)
    y = fire_module(y, squeeze=16, expand=64)
    y = layers.MaxPooling1D(pool_size=2)(y)

    y = fire_module(y, squeeze=32, expand=128)
    y = fire_module(y, squeeze=32, expand=128)
    y = layers.MaxPooling1D(pool_size=2)(y)

    y = fire_module(y, squeeze=48, expand=192)
    y = fire_module(y, squeeze=48, expand=192)
    y = fire_module(y, squeeze=64, expand=256)
    y = fire_module(y, squeeze=64, expand=256)

    y = layers.GlobalAveragePooling1D()(y)
    y = layers.Dense(num_classes, activation='sigmoid')(y)

    model = models.Model(x, y, name='squeezenet')
    return model

# Build the SqueezeNet model
model_squeezenet = build_squeezenet(input_shape=(X_train.shape[1], 1), num_classes=1)

# Compile the model
model_squeezenet.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_squeezenet = model_squeezenet.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluate the model on the test set
y_pred_squeezenet = (model_squeezenet.predict(X_test_reshaped) > 0.5).astype(int)

# Convert back to original labels
y_pred_squeezenet = label_encoder.inverse_transform(y_pred_squeezenet.flatten())
y_test_squeezenet = label_encoder.inverse_transform(y_test)

# Calculate accuracy
accuracy_squeezenet = accuracy_score(y_test_squeezenet, y_pred_squeezenet)
print(f'SqueezeNet Accuracy: {accuracy_squeezenet}')

# Calculate F1 Score
f1_squeezenet = f1_score(y_test_squeezenet, y_pred_squeezenet, pos_label='spoof')
print(f'SqueezeNet F1 Score: {f1_squeezenet}')


# Function to calculate t-DCF
def calculate_tDCF(tar_miss, tar_fa, p_tar, c_miss, c_fa):
    tDCF = p_tar * (c_miss * tar_miss + c_fa * tar_fa)
    return tDCF

# Function to calculate EER
def calculate_eer(fpr, fnr):
    eer_threshold = min(fpr[np.nanargmin(np.abs(fpr - (1 - fnr)))], fnr[np.nanargmin(np.abs(fnr - (1 - fpr)))])
    return eer_threshold

# Convert 'FAKE' and 'REAL' labels to binary labels (0 and 1)
y_test_binary = label_encoder.transform(y_test_squeezenet)

# Calculate EER and t-DCF for the SqueezeNet model
fpr_squeezenet, tpr_squeezenet, _ = roc_curve(y_test_binary, model_squeezenet.predict(X_test_reshaped).flatten())
fnr_squeezenet = 1 - tpr_squeezenet

eer_threshold_squeezenet = calculate_eer(fpr_squeezenet, fnr_squeezenet)
print(f'SqueezeNet EER: {eer_threshold_squeezenet}')

# Define the prior probabilities for the target and non-target classes
P_target = len(y_test_squeezenet[y_test_squeezenet == 'spoof']) / len(y_test_squeezenet)
P_non_target = 1 - P_target
# Define the cost parameters for t-DCF calculation
C_miss = 1  # Adjust the values based on the relative importance of miss and false alarm errors
C_fa = 1    # Adjust the values based on the relative importance of miss and false alarm errors


# Calculate t-DCF for the SqueezeNet model
tDCF_squeezenet = calculate_tDCF(fnr_squeezenet[0], fpr_squeezenet[0], P_target, C_miss, C_fa)
print(f'SqueezeNet t-DCF: {tDCF_squeezenet}')


Epoch 1/10
4540/4540 [==============================] - 94s 19ms/step - loss: 0.3308 - accuracy: 0.8984 - val_loss: 0.3292 - val_accuracy: 0.8981
Epoch 2/10
4540/4540 [==============================] - 90s 20ms/step - loss: 0.3291 - accuracy: 0.8984 - val_loss: 0.3293 - val_accuracy: 0.8981
Epoch 3/10
4540/4540 [==============================] - 91s 20ms/step - loss: 0.3289 - accuracy: 0.8984 - val_loss: 0.3293 - val_accuracy: 0.8981
Epoch 4/10
4540/4540 [==============================] - 88s 19ms/step - loss: 0.3288 - accuracy: 0.8984 - val_loss: 0.3293 - val_accuracy: 0.8981
Epoch 5/10
4540/4540 [==============================] - 92s 20ms/step - loss: 0.3289 - accuracy: 0.8984 - val_loss: 0.3293 - val_accuracy: 0.8981
Epoch 6/10
4540/4540 [==============================] - 91s 20ms/step - loss: 0.3289 - accuracy: 0.8984 - val_loss: 0.3296 - val_accuracy: 0.8981
Epoch 7/10
4540/4540 [==============================] - 88s 19ms/step - loss: 0.3288 - accuracy: 0.8984 - val_loss: 0.3295 -